<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/%20House%20Prices%3A%20Advanced%20Regression%20Techniques/House_Prices_Advanced_Regression_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from scipy.stats import norm, skew #for some statistics

path_train = 'drive/My Drive/Datasets/House Prices: Advanced Regression Techniques/train.csv'
path_test = 'drive/My Drive/Datasets/House Prices: Advanced Regression Techniques/test.csv'
train=pd.read_csv(path_train)
test=pd.read_csv(path_test)
testID = test['Id']
train = train.drop(columns=['Id'], axis=1)
test = test.drop(columns=['Id'], axis=1)


print("Train", train.shape)

train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)
train["SalePrice"] = np.log1p(train["SalePrice"])
y_train = train.SalePrice.values
train = train.drop(columns=['SalePrice'], axis=1)
all_data = pd.concat([train, test], axis=0)

all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
    
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)
all_data['OverallCond'] = all_data['OverallCond'].astype(str)
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})

skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)

all_data = pd.get_dummies(all_data)

train = all_data[:train.shape[0]]
print("train shape = ", train.shape)
test = all_data[train.shape[0]:]
print("test shape = ", test.shape)
print("Target shape = ", y_train.shape)

Train (1460, 80)
Shape all_data: (2917, 78)

Skew in numerical features: 

There are 59 skewed numerical features to Box Cox transform
train shape =  (1458, 220)
test shape =  (1459, 220)
Target shape =  (1458,)


In [12]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
print("End Lasso")
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
print("End ENet")
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
print("End KRR")
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
print("End GBoost")
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=10):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)
 

stacked_averaged_models = StackingAveragedModels(base_models = (ENet, GBoost, KRR),
                                                 meta_model = lasso)

print("End Stack")

stacked_averaged_models.fit(train.values, y_train)
stacked_train_pred = np.expm1(stacked_averaged_models.predict(train.values))
stacked_pred = np.expm1(stacked_averaged_models.predict(test.values))

print("End Stack")
model_xgb.fit(train, y_train)
xgb_train_pred = np.expm1(model_xgb.predict(train))
xgb_pred = np.expm1(model_xgb.predict(test))

print("End XGB")
model_lgb.fit(train, y_train)
lgb_train_pred = np.expm1(model_lgb.predict(train.values))
lgb_pred = np.expm1(model_lgb.predict(test.values))
print("End LGB")


End Lasso
End ENet
End KRR
End GBoost
End Stack
End Stack
End XGB
End LGB


In [13]:
print("Score RMSE = ", rmse(target, bestmodel.predict(train)))

Score RMSE =  195690.83413535455


In [104]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))
def pd2np(data, dtype=np.float32):
  return np.asarray(data, dtype=dtype)

train = pd2np(train)
from sklearn.linear_model import SGDRegressor
from sklearn import svm
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

kf = KFold(n_splits=20)
clf = SGDRegressor()
# clf = Ridge(alpha=.5)
# clf = svm.SVR()
# clf = linear_model.Lasso(alpha=0.1)
# clf = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
clf = linearmodel.Lasso(alpha =0.0005, random_state=1)
bestmodel = clf
bestscore = 10000000000
for train_index, val_index in kf.split(train):
  X_train, X_val= train[train_index], train[val_index]
  y_train, y_val = target[train_index], target[val_index]
  clf.fit(X_train, y_train)
  print("Score RMSE = ", rmse(y_val, clf.predict(X_val)))
  score = rmse(y_val, clf.predict(X_val))
  if(bestscore> score):
    bestscore = score
    bestmodel = clf
  



Score RMSE =  0.11608087956091083
Score RMSE =  0.12348797562278134
Score RMSE =  0.10641608111144478
Score RMSE =  0.08927981736122016
Score RMSE =  0.09725807145351179
Score RMSE =  0.12753668509845784
Score RMSE =  0.14299384865461556
Score RMSE =  0.11474546328885421
Score RMSE =  0.15780885065676137
Score RMSE =  0.14689832988423934
Score RMSE =  0.11124054484991198
Score RMSE =  0.10974578434984467
Score RMSE =  0.10290482850890117
Score RMSE =  0.13493822106919987
Score RMSE =  0.10642309653562951
Score RMSE =  0.10769757746974676
Score RMSE =  0.1059733774506315
Score RMSE =  0.08339598514497835
Score RMSE =  0.12930260528825785
Score RMSE =  0.1141342207706825


In [0]:
submit = pd.DataFrame()
submit['Id'] = testID
submit['SalePrice'] = stacked_pred*0.70 + xgb_pred*0.15 + lgb_pred*0.15
submit.to_csv('sub.csv', index=False)